In [3]:
import connections as con
import parser
import os
from subprocess import Popen, PIPE
import sys
import shlex
import pandas as pd

import shutil
import parser__
import pymysql
import pymysql.cursors
import json
import numpy as np

from pandas.io import sql
import MySQLdb

instagram_user =os.environ['INSTAGRAM_USER']
instagram_pass = os.environ['INSTAGRAM_PASS']

import os
sql_user =os.environ['SQL_USER']
sql_pass = os.environ['SQL_PASS']

model = parser__.ImageParser()


def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

def run_shell(cmd):
    """
    Execute the external command and get its exitcode, stdout and stderr.
    """
    args = shlex.split(cmd)

    proc = Popen(args, stdout=PIPE, stderr=PIPE,shell=True)
    out, err = proc.communicate()
    exitcode = proc.returncode
    #
    #return exitcode, out, err
    return out.decode()
    




loaded connections
Unsplash model loaded


In [4]:
def check_for_instagram_user(username):
    '''
    Run scraper to check for user name
    '''
    cmd = 'instagram-scraper ' + str(username).strip() + ' -u '+ instagram_user + ' -p ' + instagram_pass + ' --maximum 1'
    response = run_shell(cmd)
    import shutil
    try:
        shutil.rmtree(username)
        print('{} exists'.format(username))
        return True
    except :
        return False
    
def download_instagram_user(username,maximum_downloads):
    '''
    Proceed to do an actual download of the instagram user account
    '''
    cmd = 'instagram-scraper ' + str(username).strip() + ' -u '+ instagram_user + ' -p ' + instagram_pass + ' --maximum {}'.format(maximum_downloads)
    response = run_shell(cmd)
    
   
    dst = '../instagram_accounts/'+username 
    thumb_path = '../instagram_accounts/'+username +'/thumbs'
    

    
    try:  
        os.mkdir(dst)
       
    except OSError:  
        print ("Creation of the directory %s failed" % dst)
    else:  
        print ("Successfully created the directory %s " % dst)
    
    try:  
        os.mkdir(thumb_path)
       
    except OSError:  
        print ("Creation of the directory %s failed" % thumb_path)
    else:  
        print ("Successfully created the directory %s " % thumb_path)
        
        
         
    dst = '../instagram_accounts/'+username + '/' 
    copytree(username, dst)
    
    import shutil
    try:
        shutil.rmtree(username)
        print('Finished downloading')
        return True
    except :
        print('There was an issue cleaning up the username directory. Check for current access.')
        return False


# If username exists, then send an ajax callback success alert, and update user's notificatins json dict

def create_job_list(username):
    '''
    Walk through each of the files in the username search folder
    '''
   
    joblist = []
    filepath = '../instagram_accounts/'+username
    for filename in os.listdir(filepath):
        filepath_ = filepath+'/'+filename
        joblist.append(filepath_)
    return joblist
        

def execute_job_list(joblist,username):
    '''
    In a future release, this will send jobs to celery queue
    '''
    
    image_array=[]
    json_details=[]
    for job in joblist:

        output = model.predict_quality(job)
        image_array.append(job) # We need to reference the full filepaths easily here
        json_details.append(json.dumps(output))
    
    df = pd.DataFrame()
    
    df['image_filepath'] = np.array(image_array)
    df['json_details'] = np.array(json_details)
    
    connect=MySQLdb.connect(host="localhost",
                       user=sql_user,
                      passwd=sql_pass,
                       db='instagram_accounts')
    
    df.to_sql(con=connect, name=username, if_exists='replace', flavor='mysql')
    return True

#      1. In each, learn from all models. 2. Paste the json output into the user db name specific table 
#      3. If the overall score is above .95, also put it in the favorites table with username


In [5]:
username = 'joeypilgrim'

check_if_exists = check_for_instagram_user(username)
if check_if_exists==True:
    maximum_downloads = 10
    download_instagram_user(username,maximum_downloads)
    joblist = create_job_list(username)
    result = execute_job_list(joblist,username)
    
print(result)

joeypilgrim exists
Creation of the directory ../instagram_accounts/joeypilgrim failed
Creation of the directory ../instagram_accounts/joeypilgrim/thumbs failed
Finished downloading
Loaded ../instagram_accounts/joeypilgrim/49789585_2006906882942892_9076619747449110528_n.jpg
(1200,)


ValueError: operands could not be broadcast together with shapes (1,1200) (1875,) (1,1200) 